[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Arkapravaroy/managerAI/blob/main/ManagerAIV1.ipynb)


In [41]:
%%capture --no-stderr
%pip install -U langchain_openai langchain_groq langgraph trustcall langchain_core langchain_community arxiv wikipedia PyMuPDF

In [42]:
import os, getpass

def _set_env(var:str):
  # Check if the variable is set in the OS environment
  env_value = os.environ.get(var)
  if not env_value:
      # If not set, prompt the user for input
      env_value = getpass.getpass(f"{var}: ")

  # Set the environment variable for the current process
  os.environ[var] = env_value

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "manager-ai"

In [43]:
_set_env("GROQ_API_KEY")
_set_env("TAVILY_API_KEY")


## Agent's Capabilities: Preliminary list
### Product Management Specific Functionalities:
1. Market Research and Analysis: Automatically gather and analyze market data, competitor information, and industry trends to identify opportunities and inform product strategy.
2. User Feedback Analysis: Process and categorize customer feedback from various channels (e.g., support tickets, reviews, social media) to identify pain points, needs, and sentiment.
3. User Story and Requirement Drafting: Assist in drafting user stories, acceptance criteria, and product requirements based on user feedback, market analysis, and strategic goals.
4. Product Roadmap Generation and Management: Help create, manage, and update product roadmaps, suggesting prioritization based on various factors like business value, effort, and dependencies.
5. Feature Prioritization: Analyze data and requirements to suggest and facilitate the prioritization of features for development.
6. Product Performance Tracking and Analysis: Monitor key product metrics, analyze user engagement, and identify areas for improvement.

### Project Management Specific Functionalities:
1. Project Planning and Scheduling: Assist in creating detailed project plans, defining tasks, setting dependencies, estimating timelines, and generating schedules.
2. Task Management and Assignment: Automate task creation, assignment to team members based on availability and skills, and tracking of progress.
3. Resource Allocation and Optimization: Analyze project needs and resource availability to suggest optimal resource allocation and identify potential conflicts or underutilization.
4. Risk Identification and Mitigation: Proactively identify potential project risks based on historical data and current project parameters, and suggest mitigation strategies.
5. Progress Monitoring and Reporting: Automatically track project progress, update status reports, and generate dashboards for stakeholders.
6. Communication and Notification Management: Draft and send project updates, notifications, and reminders to team members and stakeholders.
7. Meeting Summarization and Action Item Extraction: Attend virtual meetings (or process transcripts) to summarize discussions and extract action items.
8. Workflow Automation: Automate repetitive project workflows, such as sending follow-up emails, updating task statuses, or triggering actions based on predefined conditions.





#### Starting building the Agent

In [44]:
from typing import TypedDict, Literal

# Update memory tool
class UpdateMemory(TypedDict):
    """ Decision on what memory type to update """
    update_type: Literal['user', 'ticket', 'instructions', 'productresearch', 'userfeedback']


**Imports**

In [45]:
import os, getpass
import uuid
from IPython.display import display, Markdown, Image
from typing import TypedDict, Literal, Optional, List, Dict, Any
from datetime import datetime
from trustcall import create_extractor # Assuming this is a local or installed lib
from pydantic import BaseModel, Field

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import ArxivLoader

from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage, BaseMessage, merge_message_runs
from langchain_core.tools import tool
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, END, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore
from langchain_groq import ChatGroq

**Model**

In [46]:
model = ChatGroq(model = "qwen-qwq-32b", temperature = 0)

**Defining Graph**

Tools for first version:


*   search internet for market research
*   User Feedback Analysis
*   extracting requirements from discussion(feature or bug or product) and call necessary tool
*   analyze bug, assign team, give tentative deadline, search internet for possible solution
*   analyze jira and categorize tasks into complete, backlog, on-time, early-finish and also track completion percentage by teams







**User Profile:**
User profile will be created so that the agent knows with whom it is talking to and then user specific conversation memory can be stored

In [47]:
class Profile(BaseModel):
  """This is the profile of the user you are having conversation with"""
  name: Optional[str] = Field(description="The user's name", default=None)
  location: Optional[str] = Field(description="The user's location", default=None)
  team: Optional[str] = Field(description="The user's team", default="management")
  designation: Optional[str] = Field(description="The user's designation", default="manager")
  email: Optional[str] = Field(description="The user's email", default=None)


class TicketDetails(BaseModel):
  """This is the details of the ticket you created by analyzing user's input and conversations. You also include details like assignee, break into pecific steps """
  task: str = Field(description="The task to be completed.")
  time_to_complete: Optional[int] = Field(description="Estimated time to complete the task (minutes).")
  deadline: Optional[datetime] = Field(
      description="When the task needs to be completed by (if applicable)",
      default=None
  )

  solutions: list[str] = Field(
      description="List of specific, actionable solutions (e.g., specific ideas, service providers, or concrete options relevant to completing the task)",
      min_items=1,
      default_factory=list
  )
  status: Literal["not started", "in progress", "done", "archived"] = Field(
      description="Current status of the task",
      default="not started"
  )

@tool
def web_search(query: str) -> Dict[str, str]: # The 'query' argument name is fine for your @tool definition
    """Search Tavily for a query and return maximum 3 results.

    Args:
        query: The search query."""
    tavily_tool = TavilySearchResults(max_results=3)
    # Invoke the Tavily tool correctly. It expects the query as the 'input'.
    # The 'query' parameter from your function signature is passed as 'input' here.
    search_results_list_of_dicts = tavily_tool.invoke(input=query) # Use 'input=query'

    # Process the results (TavilySearchResults returns a list of dictionaries)
    formatted_search_docs_list = []
    if isinstance(search_results_list_of_dicts, list):
        for doc in search_results_list_of_dicts:
            source = doc.get("url", "N/A") # Tavily results usually have 'url' and 'content'
            content = doc.get("content", "")
            formatted_search_docs_list.append(
                f'<Document source="{source}"/>\n{content}\n</Document>'
            )
    elif isinstance(search_results_list_of_dicts, str): # Sometimes it might return a string on error or simple cases
        formatted_search_docs_list.append(search_results_list_of_dicts)


    return {"web_results": "\n\n---\n\n".join(formatted_search_docs_list)}

@tool
def wiki_search(query: str) -> Dict[str, str]:
    """Search Wikipedia for a query and return maximum 2 results.

    Args:
        query: The search query."""
    # WikipediaLoader.load() is the correct way to use it.
    # The 'query' argument is part of its constructor.
    search_docs: List[Any] = WikipediaLoader(query=query, load_max_docs=2).load()
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata.get("source", "N/A")}" title="{doc.metadata.get("title", "N/A")}"/>\n{doc.page_content}\n</Document>'
            for doc in search_docs
        ])
    return {"wiki_results": formatted_search_docs}

@tool
def arxiv_search(query: str) -> Dict[str, str]:
    """Search Arxiv for a query and return maximum 3 result.

    Args:
        query: The search query."""
    # ArxivLoader.load() is the correct way.
    # The 'query' argument is part of its constructor.
    search_docs: List[Any] = ArxivLoader(query=query, load_max_docs=3).load()
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata.get("source", "N/A")}" title="{doc.metadata.get("title", "N/A")}"/>\n{doc.page_content[:2000]}\n</Document>' # Increased snippet size
            for doc in search_docs
        ])
    return {"arxiv_results": formatted_search_docs} # Corrected typo from arvix_results

# Create the Trustcall extractor for updating the user profile
profile_extractor = create_extractor(
    model,
    tools=[Profile],
    tool_choice="Profile",
)
ticket_extractor = create_extractor(
    model,
    tools=[TicketDetails],
    tool_choice="TicketDetails",
    enable_inserts=True # Important for adding multiple tickets
)

search_execution_tools = [web_search, wiki_search, arxiv_search]
search_tool_node = ToolNode(search_execution_tools)


**prompts and instructions**

In [48]:
DECIDE_ACTION_SYSTEM_PROMPT = """You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.
Your goal is to understand the user's intent and decide the best course of action.

Current User Profile:
<user_profile>
{user_profile}
</user_profile>

Current Ticket List:
<ticket>
{ticket}
</ticket>

User-defined Instructions for Ticket Management:
<instructions>
{instructions}
</instructions>

User Feedback Received So Far:
<userfeedback>
{userfeedback}
</userfeedback>

Product Research Notes:
<productresearch>
{productresearch}
</productresearch>

Based on the latest user message and the conversation history, determine your next step:
1.  **Gather Information:** If the user is asking for information that requires external knowledge (e.g., market trends, competitor analysis, specific facts, scientific papers), call the appropriate search tool: `web_search`, `wiki_search`, or `arxiv_search`.
2.  **Update Memory Directly:**
    *   If the user provides personal information (name, location, team, role), call `UpdateMemory` with `update_type: 'user'`. The specific information will be extracted from the conversation by the update node.
    *   If the user mentions a new task, bug, feature request, or something that should be tracked, call `UpdateMemory` with `update_type: 'ticket'`. The ticket details will be extracted from the conversation by the update node.
-   *   If the user specifies preferences for how you should manage or update the ToDo/Ticket list, call `UpdateMemory` with `update_type: 'instructions'`.
-   *   If the user provides general feedback, opinions, or sentiment about a product, service, or topic (not about *your* operation), call `UpdateMemory` with `update_type: 'userfeedback'`.
-   *   If the user is discussing insights, ideas, or information that should be part of ongoing product research (and doesn't require immediate new search), call `UpdateMemory` with `update_type: 'productresearch'`.
+   *   If the user specifies preferences for how you should manage or update the ToDo/Ticket list, call `UpdateMemory` using ONLY the `update_type: 'instructions'` argument. The content of the instructions will be derived from the conversation history by the update node.
+   *   If the user provides general feedback, opinions, or sentiment about a product, service, or topic (not about *your* operation), call `UpdateMemory` using ONLY the `update_type: 'userfeedback'` argument. The feedback content will be derived from the conversation history by the update node.
+   *   If the user is discussing insights, ideas, or information that should be part of ongoing product research (and doesn't require immediate new search), call `UpdateMemory` using ONLY the `update_type: 'productresearch'` argument. The research content will be derived from the conversation history by the update node.
3.  **Respond Directly:** If no tool is needed, or you need clarification, formulate a natural response to the user.

Reason carefully. If you use a tool, ensure you provide the correct arguments.
If you call `UpdateMemory`, the respective update node will handle the detailed processing and storage.

Reason carefully. If you use a tool, ensure you provide the correct arguments as specified by its schema.
If you call `UpdateMemory` for 'instructions', 'userfeedback', or 'productresearch', only provide the 'update_type' argument.
The respective update node will handle the detailed processing and storage by analyzing the conversation.

+ If the latest message in the conversation history is a ToolMessage confirming an action you just took (like a memory update),
+ your primary goal is to:
+ 1. Clearly and concisely communicate the outcome of that action to the user, incorporating the key details provided in that ToolMessage.
+ 2. Ask the user what they would like to do next, or await their next instruction.
+ 3. Avoid initiating new tool calls unless the user explicitly asks for a new action in their very next message.
+ Simply acknowledge the completed task and the information you've relayed.
"""

HANDLE_SEARCH_RESULT_SYSTEM_PROMPT = """You have received results from a search tool.
User's original intent/query that led to the search: {original_query_context}
Search Results:
<search_results>
{search_results_content}
</search_results>

Conversation History (last few messages):
{chat_history}

Now, do the following:
1.  Analyze the search results in the context of the user's query and conversation.
2.  Summarize the key findings for the user.
3.  **Crucially, decide if these findings should update any of your long-term memories:**
    *   If the findings are relevant for ongoing **product research** (e.g., market data, competitor info, trends), call `UpdateMemory` with `update_type: 'productresearch'`.
    *   If the findings represent general **user feedback/sentiment** on a topic, call `UpdateMemory` with `update_type: 'userfeedback'`.
    *   If the findings directly lead to a new actionable **task or ticket**, call `UpdateMemory` with `update_type: 'ticket'`.
    *   (Less common from search, but possible) If the findings reveal something about the user's preferences for *your* operation, call `UpdateMemory` with `update_type: 'instructions'`.
4.  If you call `UpdateMemory`, provide a brief reasoning or the synthesized data if simple. The dedicated update node will do the heavy lifting.
5.  Formulate a response to the user that includes the summary and mentions if you're updating any knowledge base. If no memory update is needed, just provide the summary.
"""

# Prompts for update nodes (can be refined further)
TRUSTCALL_INSTRUCTION = """Reflect on following interaction.
Use the provided tools to retain any necessary memories about the user.
Use parallel tool calling to handle updates and insertions simultaneously.
System Time: {time}"""

CREATE_INSTRUCTIONS_PROMPT = """Based on the entire conversation history and the user's latest messages, update the instructions for how to manage ToDo list items.
Your current instructions are:
<current_instructions>
{current_instructions}
</current_instructions>

User's input related to instructions:
{relevant_user_input}

Synthesize the new, complete set of instructions. Output only the new instructions.
"""
# You'll need similar specific prompts for update_userfeedback and update_productresearch
# For example:
UPDATE_PRODUCT_RESEARCH_PROMPT = """Based on the entire conversation history, user inputs, and any recent search results provided, update the product research notes.
Current product research notes:
<productresearch>
{current_productresearch}
</productresearch>

Relevant inputs (user messages, search summaries):
{relevant_inputs}

Synthesize the new, complete product research notes. Output only the new notes.
"""

**building nodes**

In [49]:
def load_memories(user_id: str, store: BaseStore) -> Dict[str, Any]:
    """Helper to load all memories for a user."""
    memories = {
        "user_profile": None,
        "ticket": "",
        "instructions": "",
        "userfeedback": "",
        "productresearch": ""
    }

    profile_mem = store.search(("profile", user_id))
    if profile_mem:
        memories["user_profile"] = profile_mem[0].value # Assuming Profile is stored directly

    ticket_mems = store.search(("ticket", user_id))
    # Tickets are stored as individual items, so we need to format them
    # This assumes TicketDetails.model_dump_json() was stored or similar
    formatted_tickets = []
    for mem in ticket_mems:
        if isinstance(mem.value, dict): # If stored as dict
             formatted_tickets.append(f"- Task: {mem.value.get('task', 'N/A')}, Status: {mem.value.get('status', 'N/A')}")
        elif isinstance(mem.value, str): # If stored as string (e.g. JSON string of TicketDetails)
             formatted_tickets.append(mem.value) # Or parse JSON if needed
    memories["ticket"] = "\n".join(formatted_tickets) if formatted_tickets else "No tickets yet."


    instr_mem = store.get(("instructions", user_id), "instructions")
    if instr_mem: memories["instructions"] = instr_mem.value.get("memory", "")

    feedback_mem = store.get(("userfeedback", user_id), "userfeedback")
    if feedback_mem: memories["userfeedback"] = feedback_mem.value.get("memory", "")

    research_mem = store.get(("productresearch", user_id), "productresearch")
    if research_mem: memories["productresearch"] = research_mem.value.get("memory", "")

    return memories

def decide_initial_action(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Decides the initial action: search, update memory, or respond."""
    user_id = config["configurable"]["user_id"]
    mems = load_memories(user_id, store)

    system_msg_content = DECIDE_ACTION_SYSTEM_PROMPT.format(
        user_profile=mems["user_profile"] if mems["user_profile"] else "Not yet collected.",
        ticket=mems["ticket"] if mems["ticket"] else "No tickets yet.",
        instructions=mems["instructions"] if mems["instructions"] else "None specified.",
        userfeedback=mems["userfeedback"] if mems["userfeedback"] else "None yet.",
        productresearch=mems["productresearch"] if mems["productresearch"] else "None yet."
    )

    # Get current conversation messages from state
    # The system message should ideally be the first message or managed carefully
    # For simplicity, prepend it here if not already the first message.
    # A more robust way is to ensure it's set once at the start of a session.
    conversation_messages = [SystemMessage(content=system_msg_content)] + state["messages"]
    print("conversation_messages",conversation_messages)
    response = model.bind_tools(
        [UpdateMemory, web_search, wiki_search, arxiv_search],
        # parallel_tool_calls=False # Let LLM decide if parallel makes sense
    ).invoke(conversation_messages)
    print("response",response)
    return {"messages": [response]}



# def handle_search_result(state: MessagesState, config: RunnableConfig, store: BaseStore):
#     """Processes search results and decides on next steps (e.g., update memory)."""
#     user_id = config["configurable"]["user_id"]

#     # The last message is the ToolMessage with search results
#     # The message before that is the AIMessage that called the search tool
#     # The message before that is (likely) the HumanMessage that triggered the search
#     search_tool_message = state["messages"][-1]
#     ai_call_message = state["messages"][-2] if len(state["messages"]) > 1 else None
#     user_query_message = state["messages"][-3] if len(state["messages"]) > 2 else HumanMessage(content="User query not directly available in last 3 msgs.")
#     print("search_tool_message", search_tool_message)
#     print("ai_call_message", ai_call_message)
#     print("user_query_message", user_query_message)
#     if not isinstance(search_tool_message, ToolMessage):
#         # This shouldn't happen if routed correctly
#         return {"messages": [AIMessage(content="Error: Expected search results.")]}

#     search_results_content = str(search_tool_message.content)

#     # Try to get original query context. A more robust way would be to pass this explicitly.
#     original_query_context = f"User asked: '{user_query_message.content}'"
#     if ai_call_message and ai_call_message.tool_calls:
#         original_query_context += f"\nAI decided to search for: {ai_call_message.tool_calls[0]['args']}"

#     # Prepare a limited history for the prompt to avoid excessive token usage
#     chat_history_summary = "\n".join([f"{m.type}: {m.content[:200]}..." for m in state["messages"][-5:-1]])


#     system_msg_content = HANDLE_SEARCH_RESULT_SYSTEM_PROMPT.format(
#         original_query_context=original_query_context,
#         search_results_content=search_results_content,
#         chat_history=chat_history_summary
#     )

#     # The model should summarize and optionally call UpdateMemory
#     response = model.bind_tools([UpdateMemory]).invoke([
#         SystemMessage(content=system_msg_content),
#         # Provide the search results as part of the context if needed, or rely on the prompt
#         HumanMessage(content=f"Search results received: {search_results_content[:1000]}...") # User "confirms" receipt
#     ])
#     return {"messages": [response]}


def handle_search_result(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Processes search results and decides on next steps (e.g., update memory)."""
    user_id = config["configurable"]["user_id"]
    messages = state["messages"]

    # Find the AIMessage that initiated the tool calls
    # It will be the last AIMessage before the sequence of ToolMessages
    last_ai_message_with_tool_calls = None
    for i in range(len(messages) - 1, -1, -1):
        if isinstance(messages[i], AIMessage) and messages[i].tool_calls:
            last_ai_message_with_tool_calls = messages[i]
            # All subsequent messages should be ToolMessages for these calls
            # (or an error if a tool failed before execution by ToolNode)
            tool_messages_for_this_ai_call = [m for m in messages[i+1:] if isinstance(m, ToolMessage) and m.tool_call_id in [tc['id'] for tc in last_ai_message_with_tool_calls.tool_calls]]
            break

    if not last_ai_message_with_tool_calls or not tool_messages_for_this_ai_call:
        # This shouldn't happen if routed correctly from a ToolNode
        # Or it could mean the ToolNode itself had an issue before producing ToolMessages
        print("Error: Could not find corresponding AIMessage or ToolMessages for search results.")
        return {"messages": [AIMessage(content="Error: Could not properly process search tool results.")]}

    # Get the original user query that led to this AI decision
    # Search backwards from the last_ai_message_with_tool_calls to find the preceding HumanMessage
    original_user_query_message_content = "User query not easily found."
    for i in range(messages.index(last_ai_message_with_tool_calls) -1, -1, -1):
        if isinstance(messages[i], HumanMessage):
            original_user_query_message_content = messages[i].content
            break

    # Consolidate search results
    all_search_results_content_parts = []
    original_query_context_parts = [f"User asked: '{original_user_query_message_content}'"]

    ai_decision_to_search_parts = []
    for tc in last_ai_message_with_tool_calls.tool_calls:
        tool_name = tc['name']
        tool_args = tc['args']
        ai_decision_to_search_parts.append(f"AI decided to use {tool_name} with args: {tool_args}")

    original_query_context_parts.append("\n".join(ai_decision_to_search_parts))
    original_query_context = "\n".join(original_query_context_parts)

    for tm in tool_messages_for_this_ai_call:
        # tm.content is expected to be a dictionary like {"web_results": "..."} or an error string
        if isinstance(tm.content, str) and tm.content.startswith("Error:"): # Handle tool execution errors
            all_search_results_content_parts.append(f"Error from tool {tm.name}: {tm.content}")
        elif isinstance(tm.content, dict):
            # Extract the actual results from the dict, assuming keys like 'web_results', 'wiki_results'
            for key, value in tm.content.items(): # Assumes one key-value pair with results
                 all_search_results_content_parts.append(f"Results from {tm.name} ({key}):\n{value}")
        else: # Fallback for unexpected content format
            all_search_results_content_parts.append(f"Content from tool {tm.name}: {str(tm.content)}")


    consolidated_search_results_content = "\n\n---\n\n".join(all_search_results_content_parts)

    # Prepare a limited history for the prompt
    # Use messages up to and including the last_ai_message_with_tool_calls
    history_for_prompt_idx = messages.index(last_ai_message_with_tool_calls)
    chat_history_summary = "\n".join([f"{m.type}: {str(m.content)[:200]}..." for m in messages[max(0, history_for_prompt_idx-4):history_for_prompt_idx+1]])

    system_msg_content = HANDLE_SEARCH_RESULT_SYSTEM_PROMPT.format(
        original_query_context=original_query_context,
        search_results_content=consolidated_search_results_content,
        chat_history=chat_history_summary
    )

    # The model should summarize and optionally call UpdateMemory
    response = model.bind_tools([UpdateMemory]).invoke([
        SystemMessage(content=system_msg_content),
        HumanMessage(content=f"Search results received (may include errors if tools failed): {consolidated_search_results_content[:1000]}...")
    ])
    return {"messages": [response]}

def update_userprofile(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = ("profile", user_id)

    # TrustCall expects a list of messages for context
    # The AIMessage that called UpdateMemory is state["messages"][-2]
    # The HumanMessage before that is state["messages"][-3] (if available)
    # The ToolMessage confirming UpdateMemory is state["messages"][-1] - we don't need it for extraction

    # Use the messages leading up to the UpdateMemory call
    messages_for_extraction = [m for m in state["messages"] if not isinstance(m, ToolMessage)]

    # If the UpdateMemory call had specific data, you might use that directly
    # For Profile, Trustcall will analyze history.

    trustcall_input_messages = list(merge_message_runs(
        messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat()))] +
                 messages_for_extraction
    ))

    # For Profile, we usually update a single document.
    # Get existing profile to provide to trustcall if it supports updates (not clear from snippet)
    # For simplicity, let's assume profile_extractor can infer or overwrite.
    # A more robust way: fetch existing, let trustcall generate new fields, merge them.

    existing_items = store.search(namespace)
    existing_memories = ([(existing_item.key, "Profile", existing_item.value)
                          for existing_item in existing_items]
                         if existing_items else None)

    result = profile_extractor.invoke({
        "messages": trustcall_input_messages,
        "existing": existing_memories # Pass existing if your trustcall setup uses it
    })

    # Assuming profile_extractor returns a Profile object in responses
    if result["responses"]:
        profile_data = result["responses"][0] # Assuming single profile
        # Store the Pydantic model directly, or its dict representation
        # Here, we store the model itself. Ensure your store can handle it or convert to dict.
        store.put(namespace, "user_profile_doc", profile_data) # Use a consistent key
        tool_call_id = state["messages"][-1].tool_calls[0]['id']
        confirmation_msg = f"User profile updated. Current profile details: {profile_data.model_dump_json(indent=2)}"
        return {"messages": [ToolMessage(content=confirmation_msg, tool_call_id=tool_call_id)]}
    else:
        tool_call_id = state["messages"][-1].tool_calls[0]['id']
        return {"messages": [ToolMessage(content="No profile information extracted to update.", tool_call_id=tool_call_id)]}


def update_tickets(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = ("ticket", user_id) # Store tickets under this namespace

    messages_for_extraction = [m for m in state["messages"] if not isinstance(m, ToolMessage)]

    trustcall_input_messages = list(merge_message_runs(
        messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat()))] +
                 messages_for_extraction
    ))

    # Fetch existing tickets to provide context to trustcall if it supports merging/avoiding duplicates
    existing_items = store.search(namespace)
    existing_memories = ([(existing_item.key, "TicketDetails", existing_item.value)
                          for existing_item in existing_items]
                         if existing_items else None)

    result = ticket_extractor.invoke({
        "messages": trustcall_input_messages,
        "existing": existing_memories
    })

    tool_call_id = state["messages"][-1].tool_calls[0]['id']

    # ticket_extractor with enable_inserts=True can return multiple TicketDetails
    updated_ticket_descriptions = []
    if result["responses"]:
        updated_ticket_details_for_user = []
        for r_meta, ticket_obj in zip(result["response_metadata"], result["responses"]):
            # Store each ticket. Use a unique ID for each ticket document.
            # r_meta might contain 'json_doc_id' if trustcall generates it, or use uuid
            ticket_id = r_meta.get("json_doc_id", str(uuid.uuid4()))
            store.put(namespace, ticket_id, ticket_obj.model_dump()) # Store as dict
            detail_str = (
                f"  - Task: {ticket_obj.task}\n"
                f"    Status: {ticket_obj.status}\n"
                f"    Time to complete: {ticket_obj.time_to_complete or 'N/A'} minutes\n"
                f"    Deadline: {ticket_obj.deadline.isoformat() if ticket_obj.deadline else 'N/A'}\n"
                f"    Solutions: {', '.join(ticket_obj.solutions) if ticket_obj.solutions else 'None listed'}"
            )
            updated_ticket_descriptions.append(ticket_obj.task)
            updated_ticket_details_for_user.append(detail_str)

        confirmation_msg = "Ticket(s) processed. Details:\n" + "\n".join(updated_ticket_details_for_user)
        return {"messages": [ToolMessage(content=confirmation_msg, tool_call_id=tool_call_id)]}
    else:
        return {"messages": [ToolMessage(content="No new ticket information was extracted to update/create.", tool_call_id=tool_call_id)]}
    # if updated_ticket_descriptions:
    #     return {"messages": [ToolMessage(content=f"Ticket(s) updated/created: {', '.join(updated_ticket_descriptions)}", tool_call_id=tool_call_id)]}
    # else:
    #     return {"messages": [ToolMessage(content="No ticket information extracted to update.", tool_call_id=tool_call_id)]}


def update_generic_memory(
    state: MessagesState,
    config: RunnableConfig,
    store: BaseStore,
    memory_type: Literal['instructions', 'userfeedback', 'productresearch'],
    prompt_template: str
):
    user_id = config["configurable"]["user_id"]
    namespace = (memory_type, user_id)
    key = memory_type # Use memory_type as the key for this single piece of text

    current_memory_doc = store.get(namespace, key)
    current_memory_content = current_memory_doc.value.get("memory", "") if current_memory_doc else ""

    # Use relevant parts of the conversation
    # The AI message that called UpdateMemory is state["messages"][-2]
    # The ToolMessage (this function's trigger) is state["messages"][-1]
    # User messages are before that.

    # For these text-based updates, we'll use the LLM to synthesize.
    # We need to provide it with enough context.
    # Let's give it the last few messages.
    relevant_history = state["messages"][-5:-1] # Exclude the current ToolMessage call

    # A more sophisticated approach would be to extract the *specific* user input
    # that was deemed relevant by the node calling UpdateMemory.
    # For now, we pass recent history.

    formatted_history = "\n".join([f"{m.type}: {m.content}" for m in relevant_history])

    if memory_type == "instructions":
        system_msg_content = prompt_template.format(
            current_instructions=current_memory_content,
            relevant_user_input=formatted_history # Or be more specific
        )
    elif memory_type == "productresearch":
         system_msg_content = prompt_template.format(
            current_productresearch=current_memory_content,
            relevant_inputs=formatted_history # Or pass search summary if available from UpdateMemory args
        )
    else: # userfeedback
        # Create a similar specific prompt for userfeedback
        system_msg_content = f"""Update user feedback.
        Current feedback: <current_feedback>{current_memory_content}</current_feedback>
        New inputs: <new_inputs>{formatted_history}</new_inputs>
        Synthesize new feedback notes:"""


    # The LLM should generate the new complete text for the memory
    new_memory_response = model.invoke([
        SystemMessage(content=system_msg_content),
        HumanMessage(content="Please generate the updated content based on the provided information.") # Prompt LLM to act
    ])
    new_memory_content = new_memory_response.content

    store.put(namespace, key, {"memory": new_memory_content})

    tool_call_id = state["messages"][-1].tool_calls[0]['id']
    confirmation_msg = f"{memory_type.capitalize()} memory has been updated. New content:\n---\n{new_memory_content}\n---"
    return {"messages": [ToolMessage(content=confirmation_msg, tool_call_id=tool_call_id)]}

def update_instructions(state: MessagesState, config: RunnableConfig, store: BaseStore):
    return update_generic_memory(state, config, store, "instructions", CREATE_INSTRUCTIONS_PROMPT)

def update_userfeedback(state: MessagesState, config: RunnableConfig, store: BaseStore):
    # Define a specific prompt for user feedback if needed, or adapt update_generic_memory
    USER_FEEDBACK_PROMPT = """Based on the entire conversation history and the user's latest messages, update the collected user feedback.
    Current user feedback:
    <current_feedback>
    {current_feedback}
    </current_feedback>

    User's input potentially containing feedback:
    {relevant_user_input}

    Synthesize the new, complete user feedback notes. Output only the new notes.
    """
    return update_generic_memory(state, config, store, "userfeedback", USER_FEEDBACK_PROMPT.replace("{current_feedback}", "{current_instructions}").replace("{relevant_user_input}","{relevant_user_input}")) # Quick adaptation

def update_productresearch(state: MessagesState, config: RunnableConfig, store: BaseStore):
    return update_generic_memory(state, config, store, "productresearch", UPDATE_PRODUCT_RESEARCH_PROMPT)


def route_from_initial_action(state: MessagesState) -> str:
    """Routes from decide_initial_action node based on its tool call."""
    message = state['messages'][-1]
    if not message.tool_calls:
        return END  # No tool called, so end or go to a generic response node

    tool_call = message.tool_calls[0]
    tool_name = tool_call['name']

    if tool_name == 'UpdateMemory':
        update_type = tool_call['args']['update_type']
        if update_type == 'user':
            return "update_userprofile"
        elif update_type == 'ticket':
            return "update_tickets"
        elif update_type == 'instructions':
            return "update_instructions"
        elif update_type == 'userfeedback':
            return "update_userfeedback"
        elif update_type == 'productresearch':
            return "update_productresearch"
        else:
            print(f"Warning: Unknown update_type '{update_type}' in route_from_initial_action")
            return END # Should not happen
    elif tool_name in ['web_search', 'wiki_search', 'arxiv_search']:
        # return "handle_search_result"
        return "execute_search_tools"
    else:
        print(f"Warning: Unknown tool '{tool_name}' in route_from_initial_action")
        return END # Should not happen

def route_from_search_handling(state: MessagesState) -> str:
    """Routes from handle_search_result node."""
    message = state['messages'][-1]
    if not message.tool_calls:
        return END # Search handled, no further memory update, so end

    tool_call = message.tool_calls[0]
    tool_name = tool_call['name']
    print("the tool which is called is", tool_name)
    if tool_name == 'UpdateMemory': # handle_search_result should only call UpdateMemory
        update_type = tool_call['args']['update_type']
        if update_type == 'user': # Less likely from search, but possible
            return "update_userprofile"
        elif update_type == 'ticket':
            return "update_tickets"
        elif update_type == 'instructions': # Less likely
            return "update_instructions"
        elif update_type == 'userfeedback':
            return "update_userfeedback"
        elif update_type == 'productresearch':
            return "update_productresearch"
        else:
            print(f"Warning: Unknown update_type '{update_type}' in route_from_search_handling")
            return END
    else:
        print(f"Warning: Unexpected tool '{tool_name}' from handle_search_result")
        return END


In [50]:
# --- Graph Definition ---
builder = StateGraph(MessagesState)

# Node functions will now implicitly use 'across_thread_memory' from the outer scope
def decide_initial_action_node(state: MessagesState, config: RunnableConfig): # Renamed for clarity
    return decide_initial_action(state, config, across_thread_memory)

def handle_search_result_node(state: MessagesState, config: RunnableConfig):
    return handle_search_result(state, config, across_thread_memory)

def update_userprofile_node(state: MessagesState, config: RunnableConfig):
    return update_userprofile(state, config, across_thread_memory)

def update_tickets_node(state: MessagesState, config: RunnableConfig):
    return update_tickets(state, config, across_thread_memory)

def update_instructions_node(state: MessagesState, config: RunnableConfig):
    return update_instructions(state, config, across_thread_memory)

def update_userfeedback_node(state: MessagesState, config: RunnableConfig):
    return update_userfeedback(state, config, across_thread_memory)

def update_productresearch_node(state: MessagesState, config: RunnableConfig):
    return update_productresearch(state, config, across_thread_memory)


# Add nodes
builder.add_node("decide_initial_action", decide_initial_action_node)
builder.add_node("handle_search_result", handle_search_result_node)
builder.add_node("update_userprofile", update_userprofile_node)
builder.add_node("update_tickets", update_tickets_node)
builder.add_node("update_instructions", update_instructions_node)
builder.add_node("update_userfeedback", update_userfeedback_node)
builder.add_node("update_productresearch", update_productresearch_node)
builder.add_node("execute_search_tools", search_tool_node)

# Define edges
builder.add_edge(START, "decide_initial_action")

builder.add_conditional_edges(
    "decide_initial_action",
    route_from_initial_action,
    { # Path map: output of router -> target node name
        # "handle_search_result": "handle_search_result", # REMOVE or COMMENT OUT this old line
        "execute_search_tools": "execute_search_tools",  # ADD THIS NEW LINE for search
        "update_userprofile": "update_userprofile",
        "update_tickets": "update_tickets",
        "update_instructions": "update_instructions",
        "update_userfeedback": "update_userfeedback",
        "update_productresearch": "update_productresearch",
        END: END
    }
)

builder.add_conditional_edges(
    "handle_search_result",
    route_from_search_handling,
    {
        # After search, we might update memory or end
        "update_userprofile": "update_userprofile",
        "update_tickets": "update_tickets",
        "update_instructions": "update_instructions",
        "update_userfeedback": "update_userfeedback",
        "update_productresearch": "update_productresearch",
        END: END
    }
)

# Edges from update nodes: loop back to decide_initial_action to continue conversation or end
# This allows the agent to confirm the update and wait for next user input.
builder.add_edge("update_userprofile", "decide_initial_action")
builder.add_edge("update_tickets", "decide_initial_action")
builder.add_edge("update_instructions", "decide_initial_action")
builder.add_edge("update_userfeedback", "decide_initial_action")
builder.add_edge("update_productresearch", "decide_initial_action")
builder.add_edge("execute_search_tools", "handle_search_result")


# --- Memory and Compilation ---
# Store for long-term (across-thread) memory
across_thread_memory = InMemoryStore()
# Checkpointer for short-term (within-thread) memory
within_thread_memory = MemorySaver()

graph = builder.compile(checkpointer=within_thread_memory, store=across_thread_memory)

# --- To draw the graph (optional, after !pip install pygraphviz or pyppeteer) ---
# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception as e:
#     print(f"Could not draw graph: {e}")
#     print("Mermaid representation:\n", graph.get_graph().draw_mermaid())

In [51]:
config = {"configurable": {"thread_id": "user1-session1", "user_id": "user1"}}

In [52]:
# Example 1: User provides profile info
input_messages = [HumanMessage(content="My name is Sarah, and I work for the product team in London.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

My name is Sarah, and I work for the product team in London.
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nNot yet collected.\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\nNo tickets yet.\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the conversation history, determine your next step:\n1.  **Gather Information:** If the user is asking for information that requires external knowledge (

In [31]:
# Example 2: User asks to search
input_messages = [HumanMessage(content="Can you find some recent articles on AI in project management and tell me the summary?")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

Can you find some recent articles on AI in project management and tell me the summary?
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nname='Sarah' location='London' team='product team' designation='manager' email=None\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\nNo tickets yet.\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the conversation history, determine your next step:\n1.  **Gat

In [32]:
input_messages = [HumanMessage(content="Okay, that's interesting. Let's add the point about 'AI-driven risk assessment' to our research notes.and then show me the research notes")]
config_ex3 = {"configurable": {"thread_id": "user1-session-ex3-attempt2", "user_id": "user1"}}
input_messages = [HumanMessage(content="Okay, that's interesting. Let's add the point about 'AI-driven risk assessment' to our research notes.and then show me the research notes")]

for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"): # Make sure to pass the same config
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

Okay, that's interesting. Let's add the point about 'AI-driven risk assessment' to our research notes.and then show me the research notes
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nname='Sarah' location='London' team='product team' designation='manager' email=None\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\nNo tickets yet.\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the convers

In [33]:
input_messages = [HumanMessage(content="Please create a ticket to 'Investigate new CRM options'. Assign it to me.")]
config_lance = {"configurable": {"thread_id": "lance-session2", "user_id": "Lance"}} # Use a specific user
for chunk in graph.stream({"messages": input_messages}, config_lance, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

Please create a ticket to 'Investigate new CRM options'. Assign it to me.
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nNot yet collected.\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\nNo tickets yet.\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the conversation history, determine your next step:\n1.  **Gather Information:** If the user is asking for information that requires externa

In [34]:
input_messages = [HumanMessage(content="Please show me the ticket details")]
config_lance = {"configurable": {"thread_id": "lance-session2", "user_id": "Lance"}} # Use a specific user
for chunk in graph.stream({"messages": input_messages}, config_lance, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

Please show me the ticket details
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nNot yet collected.\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\n- Task: Investigate new CRM options, Status: not started\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the conversation history, determine your next step:\n1.  **Gather Information:** If the user is asking for information that requires extern

In [53]:
input_messages = [HumanMessage(content="Please tell me what is swimming based on wikipedia and research papers?")]
config_lance = {"configurable": {"thread_id": "lance-session2", "user_id": "Lance"}} # Use a specific user
for chunk in graph.stream({"messages": input_messages}, config_lance, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    latest_message.pretty_print()
    print("-" * 30)

================================ Human Message =================================

Please tell me what is swimming based on wikipedia and research papers?
------------------------------
conversation_messages [SystemMessage(content="You are a helpful manager AI, designed to be a companion to a CEO, product manager, or executive manager.\nYour goal is to understand the user's intent and decide the best course of action.\n\nCurrent User Profile:\n<user_profile>\nNot yet collected.\n</user_profile>\n\nCurrent Ticket List:\n<ticket>\nNo tickets yet.\n</ticket>\n\nUser-defined Instructions for Ticket Management:\n<instructions>\nNone specified.\n</instructions>\n\nUser Feedback Received So Far:\n<userfeedback>\nNone yet.\n</userfeedback>\n\nProduct Research Notes:\n<productresearch>\nNone yet.\n</productresearch>\n\nBased on the latest user message and the conversation history, determine your next step:\n1.  **Gather Information:** If the user is asking for information that requires external 

**Putting the graph together**